In [0]:
!pip install -q pymorphy2 natasha pymorphy2[fast] pyphen
# !pip install -q xgboost==0.4a30 

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier, LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.grid_search import GridSearchCV
import re
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
# import xgboost as xgb
pd.options.mode.chained_assignment = None
import pyphen

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [0]:
! wget -q https://github.com/anatolyburtsev/ml_notebooks/raw/master/DMIA_2018/sport_hw0/train.csv
! wget -q https://github.com/anatolyburtsev/ml_notebooks/raw/master/DMIA_2018/sport_hw0/test.csv

In [0]:
data = pd.read_csv("train.csv")
# data["Word"] = data["Word"].apply(lambda x: x.lower())
df_majority = data[data.Label==0]
df_minority = data[data.Label==1]
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=int(df_majority.shape[0]/2),    # to match majority class
                                 random_state=123) # reproducible results
df_majority_downsampled = resample(df_majority,
                                  replace=True,
                                  n_samples=df_minority.shape[0],
                                  random_state=123)
data = pd.concat([df_majority, df_minority_upsampled])
# data = pd.concat([df_minority, df_majority_downsampled])
data = data.sample(frac=1, random_state=123)
data = data.reset_index(drop=False).drop(["index"], axis=1)

In [0]:
vowel = u'уеёыаоэяию'
consonant = u'йцкнгшщзхъфвпрлджчсмтьб'
ru_letter = vowel + consonant + u"-’'"
last_letter = list(set(data["Word"].apply(lambda x: x[-1]).get_values()))
last_2_letter = list(set(data["Word"].apply(lambda x: x[-2:]).get_values()))
last_3_letter = list(set(data["Word"].apply(lambda x: x[-3:]).get_values()))

def count_vow(word):
    vow = 0
    for i in word:
        if i in vowel:
            vow += 1
    return vow

def count_con(word):
    cons = 0
    for i in word:
        if i in consonant:
            cons += 1
    return cons

def is_only_ru(word):
    return len(set(word) - set(ru_letter)) == 0
  
def add_onotoles_old_features(data):
#     data = data[data["Word"].apply(lambda x: is_only_ru(x))==1]
    data["only_ru"] = data["Word"].apply(lambda x: is_only_ru(x))
    data["len"] = data["Word"].apply(lambda x: len(x))
    data["is_len_more_11"] = data["len"].apply(lambda x: x > 11)
    data["vow_count"] = data["Word"].apply(lambda x: count_vow(x))
    data["cons_count"] = data["Word"].apply(lambda x: count_con(x))
    data["last_vow"] = data["Word"].apply(lambda x: x[-1] in vowel)
    data["prelast_vow"] = data["Word"].apply(lambda x: len(x) > 2 and x[-2] in vowel)
    data["ohara"] = data["Word"].apply(lambda x: x[:2] == "о'")
    return data

In [0]:
data = add_onotoles_old_features(data)

In [47]:
data.head()

,Word,Label,only_ru,len,is_len_more_11,vow_count,cons_count,last_vow,prelast_vow,ohara
0,пришествию,0,True,10,False,4,6,True,True,False
1,синергиям,0,True,9,False,4,5,False,True,False
2,Сыне,0,False,4,False,2,1,True,False,False
3,Арену,0,False,5,False,2,2,True,False,False
4,Лукьянова,1,False,9,False,4,4,True,False,False


In [0]:
 # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)
# text_clf = Pipeline([('vect', CountVectorizer(analyzer='char_wb', ngram_range=(2,7), lowercase=False)),
#                      ('tfidf', TfidfTransformer(use_idf=True)),
#                      ('clf', LogisticRegression(C=5, class_weight='balanced'))
# #                      ('clf', RandomForestClassifier(max_depth=160, n_estimators=45, n_jobs=-1)),
# #                      ('clf', xgb.XGBClassifier(colsample_bytree=0.97, max_depth=10, n_estimators=90, subsample=0.97))
# #                      ('clf', SGDClassifier(alpha=1e-4, penalty='l2'))#alpha=1e-4, penalty='l2', class_weight='balanced')),
#                     ])
# print( cross_val_score(text_clf, X=X, y=y, scoring='roc_auc').mean())


In [0]:
X = data.drop(["Label"], axis=1)
y = data["Label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9)
# text_clf.fit(X_train, y_train)
# y_pred = text_clf.predict(X_test)
# print(roc_auc_score(y_pred, y_test))

In [50]:
def connect_google_drive():
  !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
  !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !apt-get update -qq 2>&1 > /dev/null
  !apt-get -y install -qq google-drive-ocamlfuse fuse
  from google.colab import auth
  auth.authenticate_user()
  from oauth2client.client import GoogleCredentials
  creds = GoogleCredentials.get_application_default()
  import getpass
  !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
  vcode = getpass.getpass()
  !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
  !mkdir -p drive
  !google-drive-ocamlfuse drive
  
connect_google_drive()

gpg: keybox '/tmp/tmpjh978h6d/pubring.gpg' created
gpg: /tmp/tmpjh978h6d/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [51]:
vect = CountVectorizer(analyzer='char_wb', ngram_range=(1,6), lowercase = False)
tfidf = TfidfTransformer(use_idf=True)
clf = LogisticRegression(C=5, class_weight='balanced')

count_transformer = vect.fit(X_train["Word"])
counts = count_transformer.transform(X_train["Word"])
tfidf_transformer = tfidf.fit(counts)
data_tfidf = tfidf_transformer.transform(counts)
clf.fit(data_tfidf, y_train)

# c_2 = count_transformer.transform(X_test)
# tfidf_2 = tfidf_transformer.transform(c_2)

# y_pred = clf.predict(tfidf_2)

LogisticRegression(C=5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [0]:
def predict_tfidf(X):
  _count = count_transformer.transform(X)
  _tfidf = tfidf_transformer.transform(_count)
  _pred = clf.predict_proba(_tfidf)
  return _pred[:, 0], _pred[:,1]

In [0]:
a = predict_tfidf(X_train["Word"])
X_train["prob_0_tfidf"] = a[0]
X_train["prob_1_tfidf"] = a[1]

In [54]:
X_train.head()

,Word,only_ru,len,is_len_more_11,vow_count,cons_count,last_vow,prelast_vow,ohara,prob_0_tfidf,prob_1_tfidf
134358,захватами,True,9,False,4,5,True,False,False,0.997915,0.002085
33583,куб.м,False,5,False,1,3,False,False,False,0.979899,0.020101
98864,Элиной,False,6,False,2,3,False,True,False,0.767338,0.232662
60141,снарядам,True,8,False,3,5,False,True,False,0.990067,0.009933
48717,краснобая,True,9,False,4,5,True,True,False,0.891603,0.108397


In [55]:
cl = LogisticRegression(C=5, class_weight='balanced')
cl.fit(X_train.drop(["Word"], axis=1), y_train)

LogisticRegression(C=5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [0]:
a = predict_tfidf(X_test["Word"])
X_test["prob_0_tfidf"] = a[0]
X_test["prob_1_tfidf"] = a[1]
pred = cl.predict(X_test.drop(["Word"], axis=1))

In [57]:
roc_auc_score(pred, y_test)

0.9185150773400854

In [0]:
# make prediction and save it to google drive
test = pd.read_csv('test.csv')

In [0]:
test = add_onotoles_old_features(test)
a = predict_tfidf(test["Word"])
test["prob_0_tfidf"] = a[0]
test["prob_1_tfidf"] = a[1]

In [111]:
test.head()

,Word,only_ru,len,is_len_more_11,vow_count,cons_count,last_vow,prelast_vow,ohara,prob_0_tfidf,prob_1_tfidf
0,Аалто,False,5,False,2,2,True,False,False,0.229065,0.770935
1,ААР,False,3,False,0,0,False,False,False,0.326859,0.673141
2,Аара,False,4,False,2,1,True,False,False,0.123993,0.876007
3,Ааре,False,4,False,2,1,True,False,False,0.502951,0.497049
4,Аарон,False,5,False,2,2,False,True,False,0.641591,0.358409


In [60]:
pred = cl.predict(test.drop(["Word"], axis=1))
print(pd.Series(pred).value_counts())
pd.Series(pred).to_csv("submission.csv", index_label=["Id","Prediction"], header=True)
! cp submission.csv drive/

0    159438
1     29482
dtype: int64


In [113]:
! wc -l submission.csv

188921 submission.csv


In [0]:
dic??

In [0]:
# dic = pyphen.Pyphen(lang='ru')

# def get_end(word):
#   variants = [x for x in dic.iterate(word)]
#   if len(variants) == 0 or len(variants[0]) == 0:
#     return ""
#   return variants[0][-1].lower()
  

# def get_endings(words):
  
#   return words.apply(lambda y: get_end(y))

In [0]:
# X_end_train = pd.get_dummies(get_endings(X_train["Word"]))
# X_end_test = pd.get_dummies(get_endings(X_test["Word"]))
# X = data.drop(["Label"], axis=1)
# y = data["Label"]
# X_end = pd.concat([X, pd.get_dummies(get_endings(X["Word"]))])
# c = LogisticRegression()
# c.fit(X_end_train, y_train)
# pred = c.predict(X_end_test)
# roc_auc_score(pred, y_test)

In [0]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9)


NameError: ignored